In [1]:
import os
os.chdir("..")
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TextClassificationPipeline,
)
import pickle
import seaborn as sns
import numpy as np
from typing import Any
import torch
import pandas as pd
import random
import re
import copy
import spacy

from engine.ner_detector import tokenize_evaluate_and_detect_NERs
from engine.data import prepare_data_for_fine_tuning, read_data
from engine.utils import get_device
from engine.result_processor import *    
device = get_device()

/home/mikolaj3/nlp-2024-fake/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available. Using GPU.


In [2]:
os.getcwd()

'/home/mikolaj3/nlp-2024-fake'

In [6]:
from pathlib import Path

models = Path("output")
models_df  = get_training_data(models, parse_accuracies=True)
models_df = models_df.loc[models_df.groupby(["model", "dataset", "training_type"])["accuracy"].idxmax(), ]
models_df

,model,dataset,training_type,run,accuracy
18,ernie,coaid,masked,1,0.980769
21,ernie,coaid,unmasked,1,0.979853
30,ernie,isot,masked,1,0.985857
33,ernie,isot,unmasked,1,0.983964
7,ernie,liar,masked,2,0.700629
11,ernie,liar,unmasked,3,0.676730
13,roberta,coaid,masked,2,0.984432
16,roberta,coaid,unmasked,2,0.981685
26,roberta,isot,masked,3,0.999332
28,roberta,isot,unmasked,2,0.999443


In [7]:
results, results_misc = process_models(models_df, device)

coaid


Map: 100%|██████████| 1092/1092 [00:00<00:00, 3016.62 examples/s]
Device set to use cuda
0it [00:00, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
1000it [02:47,  5.98it/s]
100%|██████████| 1000/1000 [00:00<00:00, 37874.90it/s]
/home/mikolaj3/nlp-2024-fake/engine/result_processor.py:39: RuntimeWarning: Mean of empty slice.
  avg_per_imp = np.array(list(map(lambda x: abs(x[1]), filter(lambda z: z[2] == 'PERSON', sentence)))).mean()
/home/mikolaj3/nlp-2024-fake/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
38it [00:06,  5.63it/s]


KeyboardInterrupt: 

In [ ]:
import json
import pickle as pkl

with open("results.json", "w") as file:
    json.dump(results, file)
    
with open('results_misc.pkl', 'wb') as file:
    pkl.dump(results_misc, file)